<a href="https://colab.research.google.com/github/nathanaelyao/CISC-474-Final-Project/blob/main/2048_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import matplotlib.pyplot as plt
import gym
!pip install git+https://github.com/rgal/gym-2048
import gym_2048

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rgal/gym-2048 to /tmp/pip-req-build-zz6lr637
  Running command git clone -q https://github.com/rgal/gym-2048 /tmp/pip-req-build-zz6lr637


In [ ]:
env = gym.make('2048-v0')
env.seed(42)


/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:20: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: int64. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:25: UserWarning: WARN: It seems a Box observation space is an image but the upper and lower bounds are not in [0, 255]. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/st

[42]

In [ ]:
def DeepQ(input_shape, num_actions):
  input = layers.Input(shape=input_shape)
  layer = layers.Dense(units=20, activation="relu", name="hidden1")(layers.Flatten()(input))
  layer = layers.Dense(units=20, activation="relu", name="hidden2")(layer)
  #layer = layers.Dense(units=256, activation="relu", name="hidden3")(layer)
  output = layers.Dense(units=num_actions, activation="linear", name="output")(layer)

  return keras.Model(inputs=input, outputs=output)

In [ ]:
# DEEP Q-LEARNING v2.0
# Parameters
episodes = 5000
discount = 0.99
epsilon = 1.0
epsilon_min = 0.01
#epsilon_max = 1.0
#epsilon_decrement = epsilon_max - epsilon_min / (episodes // 4)
epsilon_decrement = 0.995
batch_size = 32  # Batch size to be taken from replay memory

replay = []  # Replay memory, stores experiences as e=(s, a, r, s', done)
max_memory_length = 2000  # Max size of replay memory
update_t_network = 500  # Number of time steps before updating target network
warm_up = 1000  # Number of steps to run before beginning to train the policy network
num_actions = env.action_space.n  # Number of possible actions
input_shape = env.observation_space.shape  # Input shape (4, 4, 16)

ep_reward_history = [0]
high_score = 0
total_steps = 0  

optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)  # Optimizer
loss="mse"  # Loss function
p_network = DeepQ(input_shape, num_actions)  # Initialize policy network
p_network.compile(optimizer=optimizer, loss=loss)
t_network = DeepQ(input_shape, num_actions)  # Initialize target network
t_network.compile(optimizer=optimizer, loss=loss)

p_network.summary()

# Train
for ep in range(episodes):
  s = np.array(env.reset())
  ep_reward = 0
  done = False

  while not done:
    total_steps += 1
    # Select action with e-greedy
    if np.random.random() < epsilon:  # Random choice
      a = np.random.choice(num_actions)
    else:  # Greedy choice
      a = p_network.predict(np.reshape(s, (1,4,4,16)), verbose=0)
      a = np.argmax(a[0])
    
    # Epsilon decay
    # epsilon = max(epsilon - epsilon_decrement, epsilon_min)
    epsilon = max(epsilon * epsilon_decrement, epsilon_min)
    
    # Execute action and observe s', reward, and done
    s_prime, reward, done, info = env.step(a)
    s_prime = np.array(s_prime)
    ep_reward += reward
    
    # Store experience in replay memory up to the last N number of experiences
    replay.append((s,a,reward,s_prime,done))
    s = s_prime
    if len(replay) > max_memory_length:
      del replay[0]
    
    if len(replay) > batch_size:
      # Sample random batch from replay memory
      batch = random.sample(replay, batch_size)
      batch_s = np.array([e[0] for e in batch])
      batch_a = np.array([e[1] for e in batch])
      batch_rewards = np.array([e[2] for e in batch])
      batch_s_prime = np.array([e[3] for e in batch])

      targets = batch_rewards
      if not done:
        targets = batch_rewards + discount * np.amax(t_network.predict(batch_s_prime, verbose=0), axis=1)
      p_network.fit(batch_s, targets, verbose=0)

    # After x time steps, update target network with policy network weights
    if total_steps % update_t_network == 0:
      t_network.set_weights(p_network.get_weights())
    
    # Track highest score reach by any episode
    if done:
      print("Episode: {}/{}, Reward: {}, High score: {}, e: {:.2}".format(ep, episodes, ep_reward, info['highest'], epsilon))
      if info['highest'] > high_score:
        high_score = info['highest']
  
  # Track reward achieved by each episode
  ep_reward_history.append(ep_reward)

print(max(high_score))
plt.plot(ep_reward_history)
plt.title("Deep Q-Learning 2048 Rewards", fontsize='large')
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.show()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 4, 4, 16)]        0         
                                                                 
 flatten_10 (Flatten)        (None, 256)               0         
                                                                 
 hidden1 (Dense)             (None, 20)                5140      
                                                                 
 hidden2 (Dense)             (None, 20)                420       
                                                                 
 output (Dense)              (None, 4)                 84        
                                                                 
Total params: 5,644
Trainable params: 5,644
Non-trainable params: 0
_________________________________________________________________
Episode: 0/10000, Reward: 108.0, High score: 16, e: 0.99

KeyboardInterrupt: ignored

In [ ]:
# DEEP Q-LEARNING v1.0
# Parameters
episodes = 10000
discount = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_decrement = epsilon_max - epsilon_min / (episodes // 4)
batch_size = 32  # Batch size to be taken from replay memory
# Replay memory, stores s, a, r, s'
replay_s = []
replay_a = []
replay_rewards = []
replay_s_prime = []
max_memory_length = 5000  # Max size of replay memory
update_t_network = 1000  # Number of time steps before updating target network
ep_reward_history = [0]
high_score = []
total_steps = 0

num_actions = 4  # Number of possible actions
input_shape = (4, 4, 16)
p_network = DeepQ(input_shape)  # Initialize policy network
p_network.compile(optimizer="adam", loss="mse")
t_network = DeepQ(input_shape)  # Initialize target network
t_network.compile(optimizer="adam", loss="mse")
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)  # Optimizer
loss_fn = keras.losses.MeanSquaredError()  # Loss function

# Train
for ep in range(episodes):
  s = np.array(env.reset())
  ep_reward = 0
  done = False

  while not done:
    total_steps += 1
    # Select action with e-greedy
    if np.random.random() < epsilon:  # Random choice
      a = np.random.choice(num_actions)
    else:  # Greedy choice
      s_tensor = tf.convert_to_tensor(s)
      s_tensor = tf.expand_dims(s_tensor, 0)
      a = tf.argmax(p_network(s_tensor, training=False)[0]).numpy()
    
    epsilon = max(epsilon - epsilon_decrement, epsilon_min)  # Epsilon decay
    
    # Execute action and observe s', reward, and done
    s_prime, reward, done, _ = env.step(a)
    s_prime = np.array(s_prime)
    ep_reward += reward
    
    # Store experience in replay memory up to the last N number of experiences
    replay_s.append(s)
    replay_a.append(a)
    replay_rewards.append(reward)
    replay_s_prime.append(s_prime)
    s = s_prime
    if len(replay_s) > max_memory_length:
      del replay_s[:1]
      del replay_a[:1]
      del replay_rewards[:1]
      del replay_s_prime[:1]
    
    if len(replay_s) > batch_size:
      # Sample random batch from replay memory
      indices = np.random.choice(range(len(replay_s)), size=batch_size)
      batch_s = np.array([replay_s[i] for i in indices])
      batch_a = np.array([replay_a[i] for i in indices])
      batch_rewards = np.array([replay_rewards[i] for i in indices])
      batch_s_prime = np.array([replay_s_prime[i] for i in indices])

      # Pass s_primes of batch to target network
      q_values_prime = t_network.predict(batch_s_prime)
      # q_values_prime = batch_rewards + discount * tf.reduce_max(q_values_prime, axis=1)
      q_values_prime = batch_rewards + discount * tf.argmax(q_values_prime)

      p_network.fit(batch_s, q_values_prime)

      masks = tf.one_hot(batch_a, num_actions)
      with tf.GradientTape() as tape:
        # Pass batch to policy network
        q_values = p_network(batch_s)
        q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
        # Calculate loss between output and target q-values
        loss = loss_fn(q_values_prime, q_action)
      # Update weights in policy network to minimize loss, backpropagation
      grads = tape.gradient(loss, p_network.trainable_variables)
      optimizer.apply_gradients(zip(grads, p_network.trainable_variables))
    
    # After x time steps, update target network with policy network weights
    if total_steps % update_t_network == 0:
      t_network.set_weights(p_network.get_weights())

    # Track highest score reach by any episode
    if done:
      print(s)
      high_score.append(max(s))
      if ep % 500 == 0:
        print(high_score[-1])

  # Track reward achieved by each episode
  ep_reward_history.append(ep_reward)

[[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]]


ValueError: ignored